Feature Selection 
=================

In [ ]:
from copy import deepcopy
import numpy as np
import pandas as pd
import sklearn as sk
import plotnine
from plotnine import ggplot, aes, facet_wrap, scale_x_log10
plotnine.theme_set(plotnine.theme_bw())

from load_hess import hessTrain, hessTrainY, hessTest, hessTestY, probeAnnot

It is often assumed that in any one particular problem the expression
patterns of most genes---or, more generally, the values of most
features in a high-dimensional data set---are either:
-   uninformative or
-   redundant with a few maximally useful markers.

*Feature selection* attempts to identify a restricted set of such
useful features for inclusion in a classification model while
discarding the rest.

While feature selection may or may not improve the performance of a
particular classifier applied to a particular problem, it can
1.  reduce computational workload,
2.  help to avoid overfitting
    -   (though feature selection can itself be susceptible to
        overfitting!), and
    
3.  facilitate assessment of model using less high-throughput
    platforms.

A good, if somewhat dated, reference on the use of feature selection
methods in bioinformatics is [@saeys2007review], which breaks down
feature selection methods into the following three categories:

**Filter**
:   Selection done before and independently of
    classifier construction. Can be univariate or multivariate.

**Wrapper**
:   Embed classifier construction within feature
    selection process. Heuristic search methods compare models, favor
    adding or removing features based on optimization of some
    specified metric on resulting classifiers.

**Embedded**
:   Feature selection is inherently built into
    some classifier construction methods.

For a nice figure providing more detail on the advantages and
disadvantages associated with each of these categories, along with
some standard examples, you can follow the link to
<https://academic.oup.com/view-large/1739292>.

I can't do justice to the wide range of feature selection techniques
in the limited time available in this course, so we're going to focus
on one particularly simple method: a plain old $t$-test.
`sklearn.feature_selection` (`fs` here) has this built-in:
`fs.f_regression`

In order to use this feature selection method as part of a
classification "pipeline", we need to connect it ("upstream") to a
("downstream") classification algorithm:

In [ ]:
import sklearn.feature_selection as fs
import sklearn.neighbors as nbr
import sklearn.pipeline as pl
fsKnnPipeline = pl.Pipeline([
    ("featsel", fs.SelectKBest(fs.f_regression, k=25)),
    ("classifier", nbr.KNeighborsClassifier(n_neighbors=9))
])
fsKnnFit = deepcopy(fsKnnPipeline).fit(hessTrain.T, hessTrainY)
fsKnnTestPredictionProbs = fsKnnFit.predict_proba(hessTest.T)
fsKnnTestPredictionProbs[0:5, :]

In [ ]:
fsKnnTestPredictionClass = fsKnnFit.predict(hessTest.T)
fsKnnTestPredictionClass[0:5]

In [ ]:
pd.crosstab(fsKnnTestPredictionClass, hessTestY,
            rownames=["prediction"], colnames=["actual"])

We can now re-use the learner pipeline `fsKnnPipeline` for
cross-validation using `cross_val_score`:

In [ ]:
import sklearn.model_selection as ms
np.random.seed(321)
shuffle = np.random.permutation(len(hessTrainY))
trainShuffled = hessTrain.T.iloc[shuffle]
trainYShuffled = hessTrainY[shuffle]
## - set up cvScheduler to generate cross-validation folds
cvScheduler = ms.KFold(n_splits=5)
## can re-use same fsKnnPipeline object implementing ML aglorithm from above
fsKnnCvAccs = ms.cross_val_score(estimator = fsKnnPipeline,
                                 X = trainShuffled,
                                 y = trainYShuffled,
                                 cv = cvScheduler.split(trainShuffled))
np.mean(fsKnnCvAccs)

`sklearn` also supports selection of optimal parameter settings via
parameter grid search using `sklearn.model_selection.GridSearchCV`;
you don't need to create the whole grid yourself with `sklearn`, just
pass in the range of values you want for each parameter. For `Pipeline`
fits, the full parameter names are specified in
<component-name>__<within-component-feature-name> format, as shown here:

In [ ]:
 ## Cross-validated parameter grid search using sklearn:
gscv = ms.GridSearchCV(fsKnnPipeline,
                       {"featsel__k" : [10, 100, 1000, 10000],
                        "classifier__n_neighbors" : [5, 9, 19]},
                       cv = cvScheduler.split(trainShuffled))
gscv.fit(trainShuffled, trainYShuffled)
gscv.best_estimator_.named_steps["featsel"].k

In [ ]:
gscv.best_estimator_.named_steps["classifier"].n_neighbors

It is interesting to note that for this data set the size of the
selected feature set will vary considerably upon re-running the
cross-validation---but only if you change the seed passed into
`np.random.seed` before generating `shuffle`!

Feature Extraction 
==================

An alternative approach to feature selection to mitigating the
problems of overfitting and high computational workload associated
with machine learning with high-dimensional data is *feature extraction*.

While

**feature selection**
:   reduces the size of the feature set
    presented to a classification or regression algorithm by retaining
    only a small subset of the feature set,

**feature extraction**
:   applies a mathematical
    transformation to the high-dimensional input data to derive a
    low-dimensional feature set.

For example, if you were trying to classify day vs. night situations
with digital image data, you could simply average the intensities of
all pixels together to extract a "light level" feature. Note that
this single extracted feature still depends on the value of *all*
of the input features, so it doesn't reduce the amount of data you
need to collect to evaluate the model, but it does massively diminish
the complexity of the task confronting whatever downstream
classification algorithm you apply!

With gene expression data, the most obvious and widely used method of
feature extraction is PCA, so we will use this for our example. Recall
that the PC1 scores of a sample are defined as a weighted sum (or
linear combination) of feature values with the feature weights learned
so as to optimally model feature values based on (feature mean +
feature weight * sample score). Higher PCs can then be defined so as
to in a similar way so as to successively improve the model.

When building a classification or regression model using PCA for
feature extraction, we learn the feature weights for the various
principal components (which make up the elements of the "rotation
matrix"), as well as the feature mean values, using the training set
(only). These weights and means are then fixed parameters of the fit
model and should not be updated when presented with test data!

Here is a function for learning the PCs from a training set (provided
to the function as a matrix of feature values `mat`) which returns a
function `extractor` for assessing the sample scores for a test set
`newdata`:

In [ ]:
 ## arguments to extractPCs
 ## - mat is matrix or data.frame of feature values
 ## - m is number of principal component features to extract
def extractPCs(mat, m=None, *args):
     ## assume x is samples-in-rows, genes-in-columns format!
    if m is None:
        m = np.min(mat.shape)    
    mu = mat.mean(axis=0)  ## training-set estimated mean expression per gene
     ## use singular value decomposition (SVD) to compute PCs:
    svdOut = np.linalg.svd(mat - mu, full_matrices=False)
    x = svdOut[0] * svdOut[1]  ## same as R's prcomp out$x
    rotation = svdOut[2].T     ## same as R's prcomp out$rotation
    sdev = svdOut[1] / np.sqrt(len(svdOut[1])-1)  ## same as R's prcomp out$sdev
    extractor = lambda newdata : np.dot(newdata-mu, rotation[:, 0:m])
    extractor.sdev = sdev
    extractor.rotation = rotation    
    extractor.center = mu
    extractor.x = x
    extractor.m = m
     ## return the function "extractor" which can be applied to newdata;
     ## this function yields coordinates of samples in newdata in PC-space
     ## learned from the training data passed in as x argument.
    return extractor

We now need to wrap `extractPCs` function up into `sklearn`-compatible
class:

In [ ]:
class PcaExtractor(sk.base.BaseEstimator, sk.base.TransformerMixin):
    "Transforms data set into first m principal components"
    def __init__(self, m):
        self.m = m
    def fit(self, X, y=None):
        self.extractor = extractPCs(X, m=self.m)
        return self
    def transform(self, X):
        return self.extractor(X)

We can hook a `PcaExtractor` object for learning the PC features to
extract from data up to our knn classification algorithm in a manner
similar to what we did for feature selection:

In [ ]:
pcaKnnPipeline = pl.Pipeline([
    ("featextr", PcaExtractor(m=5)),
    ("classifier", nbr.KNeighborsClassifier(n_neighbors=9))
])
pcaKnnFit = deepcopy(pcaKnnPipeline).fit(hessTrain.T, hessTrainY)
pcaKnnTestPredictionClass = pcaKnnFit.predict(hessTest.T)
pd.crosstab(pcaKnnTestPredictionClass, hessTestY,
            rownames=["prediction"], colnames=["actual"])

And once again we can re-use the `pcaKnnPipeline` learner object
to do cross-validation on the training set (this is a bit slow):

In [ ]:
pcaKnnCvAccs = ms.cross_val_score(estimator = pcaKnnPipeline,
                                  X = trainShuffled,
                                  y = trainYShuffled,
                                  cv = cvScheduler.split(trainShuffled))
np.mean(pcaKnnCvAccs)

Or for a parameter grid search (this may take a while!):

In [ ]:
 ## Cross-validated parameter grid search using sklearn:
 ## (may take a while!)
gscv = ms.GridSearchCV(pcaKnnPipeline,
                       {"featextr__m" : [3, 4, 5],
                        "classifier__n_neighbors" : [5, 9, 19]},
                       cv = cvScheduler.split(trainShuffled))
gscv.fit(trainShuffled, trainYShuffled)
gscv.best_estimator_.named_steps["featextr"].m

In [ ]:
gscv.best_estimator_.named_steps["classifier"].n_neighbors

Going deeper 
============

It may have occurred to you that feature selection can be seen as a
particularly simple type of feature extraction which "transforms"
the input feature matrix by simply projecting it onto a few select
dimensions.

Similarly, just as we previously described the transformation of a
data set by PCA feature extraction as a type of prediction, we could
reverse viewpoints and frame the action of `predict` methods as
really just another type of data transformation---albeit with some
peculiar restrictions on the transformed output values (e.g., must be
probability scores, must be class labels from particular set of
possibilities, etc.).

From this point of view, an ML pipeline is an ordered sequence of ML
algorithm steps. To train such a pipeline, we go through the sequence:
-   training the $i^{\text{th}}$ ML algorithm step using the
    transformed output from step $i-1$ as our input feature matrix for
    the current step,
-   thus learning fit submodel $i$.
-   We then transform the output again using our newly trained fit
    submodel $i$ and pass it along as input feature matrix to ML
    algorithm $i+1$.

To make predictions using the fit pipeline model resulting from this
training procedure, we iterate through the the ordered sequence of
trained submodels, taking the transformed output from step $i-1$ as
input feature matrix to be transformed by fit submodel $i$ and then
passed along to step $i+1$. The predicted values are then whatever is
output from the final step of the fit pipeline.

The field of *deep learning*
([@goodfellow2016deep]) builds such pipelines out of individual
steps ("layers") for which the argument feature matrix and output
transformed feature matrix are similar enough in nature that the same
type of submodel can be linked together repeatedly to generate very
long pipelines. Because each individual layer in a deep learning model
is itself generally composed of many similar subunits (artificial
"neurons"), the structure of a deep learning model is typically
referred to as a *network* instead of a pipeline, and we speak of
a many-layer network as being *deep* instead of long.

Deep learning is beyond the scope of this course, but if you work on
any projects involving machine learning long enough it's bound to come
up at some point. Especially in problems with very large numbers of
$n$ sampling units, deep learning models can often outperform other
methods, though they are prone to overfitting and tend to require a
great deal of time and effort to get working correctly.